# Datasource

In [2]:
from dateutil.parser import parse
import dateutil
import logging
import os
import json
import socket
import requests
import time
import re
import urllib
from retrying import retry
import pprint

from googleapiclient.errors import HttpError
from googleapiclient import discovery
from oauth2client import GOOGLE_TOKEN_URI
from oauth2client.client import GoogleCredentials, OAuth2Credentials
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from googleapiclient.errors import HttpError

#from auth import DEFAULT_API_ROOT
from energyworx_public import domain, rule
from energyworx_public.base import EnergyworxDomain, StructuredProperty, Property, EnumProperty, DateTimeProperty, MappingStructuredProperty, MappingProperty, MappingEnumProperty
from energyworx_public.enums import MappedFieldType, TransformationMapFunctionType, TagType, UnitType, VirtualDatasourceAggregationType, DatasourceType, DatapointType, TimeslicePeriodType
from energyworx_client import client
#from tags import parse_tags
#from timeseries import parse_result_df, parse_nested_structure_result_df

logger = logging.getLogger()
pp = pprint.PrettyPrinter(1)
print('imported modules.')

imported modules.


## Google Oauth Quickstart

In [3]:
base = os.getcwd()
print('working in {}.'.format(base))

working in /media/steven/big_boi/la-tools-test/ewx_client.


In [4]:
with open('client_secret_apps.googleusercontent.com.json', 'r') as f:
    secret = json.load(f)
    secret = secret['web']

### client secret json

In [5]:
secret

{'client_id': '71887862485-2e752er8kmsjns39idcb8i0819biji4q.apps.googleusercontent.com',
 'project_id': 'ewx-api-access',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'token_uri': 'https://oauth2.googleapis.com/token',
 'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'client_secret': 'Mn6oFIAq6vyrv39EjMso0kVo',
 'redirect_uris': ['https://la-tools-engie.southcentralus.cloudapp.azure.com/hub/oauth_callback'],
 'javascript_origins': ['http://52.228.24.147',
  'https://la-tools-engie.southcentralus.cloudapp.azure.com']}

## Google Oauth2

In [6]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

flow = InstalledAppFlow.from_client_secrets_file('client_secret_apps.googleusercontent.com.json', SCOPES)
flow.redirect_uri = secret['redirect_uris'][0]
authorization_url, state = flow.authorization_url(access_type='offline', include_granted_scopes='true')

response = requests.get(authorization_url)
print('returned status code: {}.'.format(response.status_code))
print(authorization_url)
#creds = flow.run_console()

returned status code: 200.
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=71887862485-2e752er8kmsjns39idcb8i0819biji4q.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fla-tools-engie.southcentralus.cloudapp.azure.com%2Fhub%2Foauth_callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=J46EEnovdNbCkWyPEiP8Vx7OCxaTja&access_type=offline&include_granted_scopes=true


In [7]:
creds

NameError: name 'creds' is not defined

### input response url

In [ ]:
redirect_response = input('paste url here: ')

In [11]:
print(redirect_response)

https://la-tools-engie.southcentralus.cloudapp.azure.com/hub/oauth_callback?state=MayCeY37tbOQP307iwN4X200Ll6Fbx&code=4/vAHnaaUCDm5FGINhFV8ylKiSyFrS9-ll8w5wmBdk7a4HjkdtwfXRRCTIYZ5IZlth9M4FWzqROe0gNs30SqVBEzQ&scope=https://www.googleapis.com/auth/gmail.readonly%20openid&authuser=0&session_state=d3e1e8b66f31ef53a250698d8536fa289b5d8993..50e6&prompt=consent


In [13]:
auth = {'state':'MayCeY37tbOQP307iwN4X200Ll6Fbx', 'code':'4/vAHnaaUCDm5FGINhFV8ylKiSyFrS9-ll8w5wmBdk7a4HjkdtwfXRRCTIYZ5IZlth9M4FWzqROe0gNs30SqVBEzQ',
       'scope':'https://www.googleapis.com/auth/gmail.readonly%20openid&authuser=0','session_state': 'd3e1e8b66f31ef53a250698d8536fa289b5d8993..50e6'}
pp.pprint(auth)

{'code': '4/vAHnaaUCDm5FGINhFV8ylKiSyFrS9-ll8w5wmBdk7a4HjkdtwfXRRCTIYZ5IZlth9M4FWzqROe0gNs30SqVBEzQ',
 'scope': 'https://www.googleapis.com/auth/gmail.readonly%20openid&authuser=0',
 'session_state': 'd3e1e8b66f31ef53a250698d8536fa289b5d8993..50e6',
 'state': 'MayCeY37tbOQP307iwN4X200Ll6Fbx'}


In [15]:
auth = requests.post('https://oauth2.googleapis.com/token', params = auth)

In [16]:
auth.content

b'{\n  "error": "unsupported_grant_type",\n  "error_description": "Invalid grant_type: "\n}'

### basic get request

In [ ]:
r = requests.get('https://www.googleapis.com/oauth2/v1/userinfo', creds)
print(r.content)

## EWX API

In [3]:
class EWX(object):

    def __init__(self, namespace_id=None, api_root=None, credentials=None):
        super(EWX, self).__init__()
        if not namespace_id:
            namespace_id = os.environ.get('EWX_NAMESPACE')
            logger.warning("No namespace parameter could be found, so we are using the primary namespace of the user")
        if not api_root:
            api_root = DEFAULT_API_ROOT
        self.namespace_id = namespace_id
        self.api_root = api_root
        logger.info('Using namespace = %s and api_root = %s', namespace_id, api_root)
        #self.client = ExternalService.get_client(api_root=api_root, credentials=credentials)

        
    def __update_namespace(self, request, asynch=False):
        request.headers.update({'x-namespace': self.namespace_id})
        if asynch:
            request.headers.update({'x-async-request': asynch})
        return request

    def __execute_and_handle_response(self, request, asynch=False):
        try:
            updated_request = self.__update_namespace(request, asynch)
        except Exception as ex:
            logger.error('Namespace could not be updated: %s', ex, exc_info=True)
            raise Exception('Namespace could not be updated: {}'.format(ex))
        for count in range(2):
            try:
                return updated_request.execute(num_retries=3)
            except socket.error as socket_error:
                if socket_error.strerror == 'Connection reset by peer':
                    logger.info('%s -> retry', socket_error.strerror)
                    continue
            except HttpError:
                raise
            except Exception as ex:
                logger.error('Request failed: %s', ex, exc_info=True)
                raise Exception('Request failed: {}'.format(ex))

    def upload_file(self, filename, file_content=None, tags=None, adapter_id=None, streaming=False):
        """It is important to use the exact same way of uploading data when developing, so this function
        uploads files using the EDC way. First we get the upload url to blobstore via the API (using the
        EWX client) and then we upload to blobstore API. The prepare process is then triggered by GAE
        when a market adapter id is given.

        Args:
            filename (str): The filename to use when uploading
            file_content (str): The file contents to upload
            tags (list[str]): Tags that needs to be assigned to the file to be uploaded so it can be found in the filemanager
            adapter_id (str): The adapter id, if given, after the upload an ingest will be triggered with this market adapter id
            streaming (bool): When set to true, it will use the streaming pipeline

        Returns:
            object
        """
        def _retry_if_exception(exception):
            """ Specify an exception you need. or just True"""
            return isinstance(exception, RuntimeError)
            
        STOP_MAX_DELAY = 600000
        WAIT_EXPONENTIAL_MAX = 10000
        WAIT_EXPONENTIAL_MULTIPLIER = 1000
        @retry(retry_on_exception=_retry_if_exception, wait_exponential_multiplier=WAIT_EXPONENTIAL_MULTIPLIER, wait_exponential_max=WAIT_EXPONENTIAL_MAX, stop_max_delay=STOP_MAX_DELAY)
        def _do_upload(upload_url, files, data):
            logging.info("Using upload_url: %s", upload_url)
            response = requests.post(upload_url, files={filename: file_content},
                                     data=dict(adapter_id=adapter_id, streaming=streaming),
                                     headers={'X-NAMESPACE': self.namespace_id, 'Accept': 'application/json,application/vnd.ewx.v2'})
            if 200 <= response.status_code < 300:
                logging.info('File %s successfully uploaded', filename)
                return response
            else:    
                logging.error('File %s could not be uploaded. Error: %s %s', filename, response.status_code, response.reason)
                raise RuntimeError('File {} could not be uploaded. Error: {} {}'.format(filename, response.status_code, response.reason))
                
        if not file_content:
            raise RuntimeError("File content is required!")
        if tags is None:
            tags = []
        logger.info("Uploading %s to blobstore with size %s", filename, len(file_content))
        res = self.get_upload_url(tags=tags, market_adapter_id=adapter_id, use_streaming=streaming)
        if not res:
            raise RuntimeError("Did not get a valid response for upload url")
        upload_url = res.get('uploadUrl')
        if not upload_url:
            raise RuntimeError("Could not create an uploadUrl with filename %s, and adapter_id %s", filename, adapter_id)
        return _do_upload(upload_url, files={filename: file_content}, data=dict(adapter_id=adapter_id, streaming=streaming))

    def download_file(self, blob_key):
        request_url = self.api_root + '/files/get/' + blob_key
        credentials = ExternalService._get_credentials(None)
        import httplib2
        http = httplib2.Http(disable_ssl_certificate_validation=True, timeout=60)
        credentials.authorize(http)
        response, content = http.request(request_url, method='GET', headers={
            'X-NAMESPACE': self.namespace_id,
            'Accept': 'application/json,application/vnd.ewx.v2',
        })

        if 200 <= response.status < 300:
            return content
        else:
            raise RuntimeError('File download error: {} {}'.format(response.status, response.reason))
            
            
    def get_datasource(self, id):
        """ Gets a datasource by identifier.

        Args:
            id (str): A datasource identifier as a string.

        Returns:
            dict: A datasource dictionary.
        """
        request = self.client.datasource().datasource().get(id=id)
        return self.__execute_and_handle_response(request)

      # Query
    def execute_query(self, query, job_id=None, limit=10, page_token=None, raw_result=False, priority='interactive'):
        """ Execute an EQL query.

        Args:
            query (str): The query as a string.
            job_id (str): if provided, it will try to fetch the result from this job id
            limit (int):
            page_token (str or None):
            raw_result (bool): return raw JSON result from API or parse into dataframe

        Returns:
            str or dict or pd.DataFrame: Results of the query in a dataframe.
        """
        job_complete = False
        while not job_complete:
            request = self.client.query().query().execute(query=query, jobId=job_id, limit=limit, pageToken=page_token, priority=priority)
            result = self.__execute_and_handle_response(request)
            job_complete = result['reference'].get('jobComplete', True)
            job_id = result['reference'].get('jobId')
            time.sleep(0.1)
        if isinstance(result, str) or raw_result:
            return result
        try:
            return parse_result_df(result=result)
        except Exception:
            pass
        try:
            return parse_nested_structure_result_df(result=result)
        except Exception:
            pass
        try:
            return parse_tags(result=result)
        except Exception:
            pass

    def search_files(self, filename=None, tags=None, read_only=None, created_date=None, user_id=None, market_adapter_id=None, limit=None, page_token=None):
        request = self.client.storage().storage().files().search(filename=filename, tags=tags,
                                                                 readOnly=read_only, createdDate=created_date,
                                                                 userId=user_id, marketAdapterId=market_adapter_id,
                                                                 limit=limit, pageToken=page_token)
        return self.__execute_and_handle_response(request)

    def ingest_files(self, market_adapter_id=None, use_streaming=False, file_locations=None):
        request = self.client.storage().storage().files().ingest(body=dict(marketAdapterId=market_adapter_id, useStreaming=use_streaming,
                                                                 fileLocations=file_locations))
        return self.__execute_and_handle_response(request)

    

## init

In [ ]:
ewx = EWX('na.engie.com', 'https://ewx-live.appspot.com/_ah/api/', creds)